# Classification Cleaning

In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
from math import factorial, sqrt, fabs, floor, isnan
import datetime
import re
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

# Reading Data and Exploring Features

In [7]:
latlonabrev = pd.read_csv("./data/state_latlon.csv")
abrevDF = pd.read_csv("./data/state_abrev.csv")
latlonDF = pd.merge(latlonabrev,abrevDF,on="ABBREVIATION")
latlonDF = latlonDF.drop('ABBREVIATION',axis = 1)
#print latlonDF.head()

In [8]:
details = pd.DataFrame()
pathdate1 = "_c20160223.csv"
pathdate2 = "_c20160419.csv"
for year in range(1996, 2016):
    path = "./data/StormEvents_details-ftp_v1.0_d" + str(year)
    #path = "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d" + str(year)
    if year <= 2013:
        path += "_c20160223.csv"
    else:
        path += "_c20160419.csv"
    print path
    df = pd.read_csv(path,
                   dtype = {'BEGIN_TIME' : str,
                            'BEGIN_YEARMONTH' : str,
                            'BEGIN_DATE_TIME' : str,
                            'END_TIME' : str,
                            'END_YEARMONTH' : str,
                            'END_DATE_TIME' : str,
                            'BEGIN_AZIMUTH' : str,
                            'END_AZIMUTH' : str,
                            'FLOOD_CAUSE' : str,
                            'TOR_OTHER_WFO' : str,
                            'TOR_OTHER_CZ_STATE' : str,
                            'TOR_OTHER_CZ_NAME' : str,
                            'DAMAGE_CROPS' : str,
                            'DAMAGE_PROPERTY' : str,
                            'SOURCE' : str,
                            'MAGNITUDE' : float,
                            'MAGNITUDE_TYPE': str
                           })
    details = details.append(df, ignore_index=True)

./data/StormEvents_details-ftp_v1.0_d1996_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d1997_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d1998_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d1999_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2000_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2001_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2002_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2003_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2004_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2005_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2006_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2007_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2008_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2009_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2010_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2011_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2012_c20160223.csv
./data/StormEvents_details-ftp_v1.0_d2013_c20160

In [9]:
# TIMES
details['BEGIN_DT'] = pd.to_datetime(details.BEGIN_DATE_TIME, 
                                     format="%d-%b-%y %H:%M:%S", errors = 'coerce')
details['END_DT'] = pd.to_datetime(details.END_DATE_TIME, 
                                   format="%d-%b-%y %H:%M:%S", errors = 'coerce')
details['BEGIN_DAY'] = details['BEGIN_DT'].map(lambda x: x.day)
# 0 -- Monday, 6 -- Sunday
details['BEGIN_WEEKDAY'] = details['BEGIN_DT'].map(lambda x: x.weekday())
details['BEGIN_MONTH'] = details['BEGIN_DT'].map(lambda x: x.month)
details['BEGIN_YEAR'] = details['BEGIN_DT'].map(lambda x: x.year)
details['BEGIN_HOUR'] = details['BEGIN_DT'].map(lambda x: x.hour)
details['END_DAY'] = details['END_DT'].map(lambda x: x.day)
details['END_WEEKDAY'] = details['END_DT'].map(lambda x: x.weekday())
details['END_MONTH'] = details['END_DT'].map(lambda x: x.month)
details['END_YEAR'] = details['END_DT'].map(lambda x: x.year)
details['END_HOUR'] = details['END_DT'].map(lambda x: x.hour)
details['DURING_WEEKEND'] = (details['BEGIN_WEEKDAY'] >= 5) | (details['END_WEEKDAY'] >= 5)
details['BEGIN_MORNING'] = (details['BEGIN_HOUR'] <= 8) 
details['BEGIN_DAY'] = ((details['BEGIN_HOUR'] > 8) & (details['BEGIN_HOUR'] <= 16)) 
details['BEGIN_EVENING'] = (details['BEGIN_HOUR'] > 16) 

details['DURATION'] = details['END_DT'] - details['BEGIN_DT']
details['DURATION_SECONDS'] = details['DURATION'].map(lambda x: x.total_seconds())

In [10]:
# Ignore observations with BEGIN timestamp > END timestamp
# Ignore observations with durations > 3 months
# 43 observations 
details = details.loc[details.BEGIN_DT <= details.END_DT, :]
details = details.loc[details.DURATION < datetime.timedelta(seconds = 7776000)]

In [11]:
seasons_dict = {
    3: 'spring', 4: 'spring', 5: 'spring',
    6: 'summer', 7: 'summer', 8: 'summer',
    9: 'fall', 10: 'fall', 11: 'fall',
    12: 'winter', 1: 'winter', 2: 'winter'
}
details['SEASON'] = details.BEGIN_MONTH.map(lambda x: seasons_dict[x])

In [12]:
# DURATION measured in units of days
details['DURATION'] = details.DURATION_SECONDS/86400.0
details['DURATION_HOURS'] = details.DURATION_SECONDS.map(lambda x: int(floor(x/3600.0)))

In [13]:
def money_to_int(s):
    money_dict = { 'K' : 1e3, 'k' : 1e3, 'M' : 1e6, 'm' : 1e6, 
                   'G' : 1e9, 'g' : 1e9, 'T' : 1e12, 't' : 1e12,
                   'B' : 1e9, 'b' : 1e9}
    try:
        if len(s) == 1:
            dec = money_dict[s]
        else:
            dec = money_dict[re.split('[0-9.]+',s)[1]]
    except:
        try:
            return float(s)
        except:
            formats.append(s)
            return None
    try:
        num = float(re.split('[a-z]+', s, flags=re.IGNORECASE)[0])
        return num*dec
    except:
        return dec

In [14]:
details['DAMAGE_CROPS'] = details['DAMAGE_CROPS'].map(lambda x: money_to_int(x))
details['DAMAGE_PROPERTY'] = details['DAMAGE_PROPERTY'].map(lambda x: money_to_int(x))

In [15]:
# About 52% of the observations has missing DAMAGE_CROPS values.
# About 42% of the observations has mising DAMAGE_PROPERTY values.
print 1.0*sum(details.DAMAGE_CROPS.isnull())/len(details)
print 1.0*sum(details.DAMAGE_PROPERTY.isnull())/len(details)

0.520487669754
0.420655115347


In [16]:
# Adjusting for inflation: 1996 - 2016
# http://data.bls.gov/cgi-bin/cpicalc.pl
inflation_dict = {
                  1996 : 1.52,
                  1997 : 1.48,
                  1998 : 1.46,
                  1999 : 1.43,
                  2000 : 1.38,
                  2001 : 1.34,
                  2002 : 1.32,
                  2003 : 1.29,
                  2004 : 1.26,
                  2005 : 1.22,
                  2006 : 1.18,
                  2007 : 1.15,
                  2008 : 1.11,
                  2009 : 1.11,
                  2010 : 1.09,
                  2011 : 1.06,
                  2012 : 1.04,
                  2013 : 1.02,
                  2014 : 1.01,
                  2015 : 1.00,
                  2016 : 1.00
                 }
details['INFLATION'] = details['BEGIN_YEAR'].map(lambda x: inflation_dict[x])
details['DAMAGE_CROPS'] = details['DAMAGE_CROPS']*details['INFLATION']
details['DAMAGE_PROPERTY'] = details['DAMAGE_PROPERTY']*details['INFLATION']

In [17]:
# Setting LAT, LON according to latlonDF for missing LAT, LON values
latlonDF.STATE = latlonDF.STATE.str.lower()
details.STATE = details.STATE.str.lower()
details = pd.merge(details,latlonDF,on='STATE')
details.loc[details.BEGIN_LAT.isnull(),'BEGIN_LAT'] = \
    details.loc[details.BEGIN_LAT.isnull(),'LATITUDE']
details.loc[details.BEGIN_LON.isnull(),'BEGIN_LON'] = \
    details.loc[details.BEGIN_LON.isnull(),'LONGITUDE']
details = details.drop(['LATITUDE','LONGITUDE'],axis=1)

In [18]:
# LAT, LON measured in units of 1/100 of LAT, LON
details['LAT'] = details.BEGIN_LAT/100.0
details['LON'] = details.BEGIN_LON/100.0

In [19]:
subset_all = details[['BEGIN_HOUR',
                    'BEGIN_YEAR',
                    'SEASON',
                    'DURATION_HOURS',
                    'DURATION',
                    'DURING_WEEKEND',
                    'BEGIN_MORNING',
                    'BEGIN_DAY',
                    'BEGIN_EVENING',
                    'STATE',
                    'EVENT_TYPE',
                    'WFO',
                    'INJURIES_DIRECT',
                    'INJURIES_INDIRECT',
                    'DEATHS_DIRECT',           
                    'DEATHS_INDIRECT',         
                    'DAMAGE_PROPERTY',        
                    'DAMAGE_CROPS',
                    'MAGNITUDE',
                    'CATEGORY',
                    'TOR_F_SCALE',
                    'BEGIN_RANGE',
                    'BEGIN_AZIMUTH',
                    'END_RANGE',
                    'END_AZIMUTH',
                    'LAT',
                    'LON'
                   ]]
print subset_all.dtypes

BEGIN_HOUR             int64
BEGIN_YEAR             int64
SEASON                object
DURATION_HOURS         int64
DURATION             float64
DURING_WEEKEND          bool
BEGIN_MORNING           bool
BEGIN_DAY               bool
BEGIN_EVENING           bool
STATE                 object
EVENT_TYPE            object
WFO                   object
INJURIES_DIRECT        int64
INJURIES_INDIRECT      int64
DEATHS_DIRECT          int64
DEATHS_INDIRECT        int64
DAMAGE_PROPERTY      float64
DAMAGE_CROPS         float64
MAGNITUDE            float64
CATEGORY             float64
TOR_F_SCALE           object
BEGIN_RANGE          float64
BEGIN_AZIMUTH         object
END_RANGE            float64
END_AZIMUTH           object
LAT                  float64
LON                  float64
dtype: object


## Storm Categories

In [20]:
def storm_def(s):
    if re.search('[Ww]ind|[Dd]ust', s):
        return 'Wind'
    if re.search('[Ss]torm|Tropical Depression|[Ll]ightning|[Hh]ail|[Rr]ain|[Ff]og|[Bb]lizzard', s):
        return 'Storm'
    if re.search('Northern Lights', s):
        return 'Storm'
    if re.search('[Tt]ornado|Funnel Cloud|[Ww]aterspout', s):
        return 'Tornado'
    if re.search('[Hh]urricane', s):
        return 'Hurricane'
    if re.search('[Ff]lood|Rip Current|Debris Flow|[Ll]andslide', s):
        return 'Flood'
    if re.search('[Hh]eat|[Aa]sh|[Dd]rought|[Ff]ire|[Ss]moke', s):
        return 'Heat'
    if re.search('[Ss]now|[Aa]valanche|[Ww]inter|[Ff]rost|[Ff]reez|[Ss]leet', s):
        return 'Winter Weather'
    if re.search('[Tt]ide|[Tt]sunami|[Ss]urf|[Ww]ave|[Ss]eiche', s):
        return 'Tide'
    print s
    return None

In [21]:
subset_all['EVENT_CATEGORY'] = subset_all['EVENT_TYPE'].map(lambda x: storm_def(x))
print subset_all.groupby('EVENT_CATEGORY').size().sort_values(ascending=False)
subset_all = subset_all.loc[ subset_all.EVENT_CATEGORY.notnull(),:]

OTHER
EVENT_CATEGORY
Storm             384624
Wind              366671
Flood             115248
Winter Weather    112933
Heat               77756
Tornado            36174
Tide                7718
Hurricane           1729
dtype: int64


/home/lucia/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Dealing with missing values

In [22]:
# No good way to substitute for nans in MAGNITUDE -- too many data points with nan
# No good way to substitute for nans in CATEGORY  -- too many data points with nan

# Setting TOR_F_SCALE nan values to 'N/A'
subset_all.loc[subset_all.TOR_F_SCALE.isnull(),'TOR_F_SCALE'] = 'N/A'

## Counting impact of storms

In [23]:
def get_yes(x):
    if isnan(x):
        return None
    if x == 0.0:
        return int(0)
    else:
        return int(1)
    
subset_all['INJURIES'] = subset_all['INJURIES_DIRECT'] + subset_all['INJURIES_INDIRECT']
subset_all['DEATHS'] = subset_all['DEATHS_DIRECT'] + subset_all['DEATHS_INDIRECT']
subset_all['INJURIES_YES'] = subset_all.INJURIES.map(lambda x: int(x != 0))
subset_all['DEATHS_YES'] = subset_all.DEATHS.map(lambda x: int(x != 0))
subset_all['DAMAGE_PROPERTY_YES'] = subset_all.DAMAGE_PROPERTY.map(lambda x: get_yes(x))
subset_all['DAMAGE_CROPS_YES'] = subset_all.DAMAGE_CROPS.map(lambda x: get_yes(x))

In [24]:
subset_all.to_pickle('./dfs/subset_all')

In [25]:
subset_trainvalid, subset_test = train_test_split(subset_all, test_size = 0.2,
                                                  random_state=123)
subset_trainvalid.to_pickle('./dfs/subset_trainvalid')
subset_test.to_pickle('./dfs/subset_test')